In [20]:
import sys, os
sys.path.insert(0, 'D:\Senior Year Northeastern University\DS Capstone\ImageOutpainting\model')
sys.argv[1] = "pix2pix"
#print(sys.path)

import torch
from PIL import ImageColor, Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from train import main
from masking.mask_transforms import Datum, ApplyMaskTransform, RandomRightTransform, MaskedAreaTransform
from masking.mask_gen import RightStripeMaskGenerator, TopStripeMaskGenerator
from config import config

In [21]:
gen = main()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--num_epochs NUM_EPOCHS] [--unet_lr UNET_LR] [--disc_lr DISC_LR]
                             [--gen_lr GEN_LR]
ipykernel_launcher.py: error: unrecognized arguments: pix2pix C:\Users\jayma\AppData\Roaming\jupyter\runtime\kernel-9001e483-ec98-4842-874d-c768813e4966.json


SystemExit: 2

In [ ]:
class Utils:
    def show(self, img):
        toPIL = transforms.ToPILImage()
        img = toPIL(img[:, :3, :, :][0])
        plt.imshow(img)
u = Utils()

In [ ]:
class RightImageLoader:
    def __init__(self, filepaths):
        self.filepaths = filepaths
    
    def __iter__(self):
        for fp in self.filepaths:
            to_yield = Image.open(fp), None
            yield to_yield
            
class LeftImageLoader:
    def __init__(self, filepaths):
        self.filepaths = filepaths
    
    def __iter__(self):
        for fp in self.filepaths:
            to_yield = Image.open(fp), None
            yield to_yield
            
class TopImageLoader:
    def __init__(self, filepaths):
        self.filepaths = filepaths

    def __iter__(self):
        for fp in self.filepaths:
            x, y = Image.open(fp), Image.open(fp)
            
            both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = -0.1)
            ])

            xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
            yonly = MaskedAreaTransform(config)
            p = both(x)
            x_, y_ = xonly(p), yonly(p)
            print(x_.shape, y_.shape)
            
            yield x, y

In [ ]:
class OutsideRightDataset(torch.utils.data.IterableDataset):
    
    def __init__(self):
        super(OutsideRightDataset).__init__()
        self.dataset = self.load_dataset()
                
    def __iter__(self):
        for x, _ in iter(self.dataset):
            both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = 1.1)
            ])

            xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
            yonly = MaskedAreaTransform(config)
            p = both(x)
            yield xonly(p), yonly(p)
            
    def __len__(self):
        return len(self.train_dataset)
    
    def get_pic_filepaths(self, directions):
        for direction in directions:
            for pic in os.listdir(f"OutsideData/Selected Options/{direction}"):
                yield os.path.join(f"OutsideData/Selected Options/{direction}", pic)
                
    def load_dataset(self):
        directories = ["Right"]
        filepaths = list(self.get_pic_filepaths(directories))
        return RightImageLoader(filepaths)

In [ ]:
class OutsideLeftDataset(torch.utils.data.IterableDataset):
    
    def __init__(self):
        super(OutsideLeftDataset).__init__()
        self.dataset = self.load_dataset()
                
    def __iter__(self):
        for x, _ in iter(self.dataset):
            both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.RandomHorizontalFlip(1.0),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = 1.1)
            ])

            xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
            yonly = MaskedAreaTransform(config)
            p = both(x)
            yield xonly(p), yonly(p)
            
    def __len__(self):
        return len(self.train_dataset)
    
    def get_pic_filepaths(self, directions):
        for direction in directions:
            for pic in os.listdir(f"OutsideData/Selected Options/{direction}"):
                yield os.path.join(f"OutsideData/Selected Options/{direction}", pic)
                
    def load_dataset(self):
        directories = ["Left"]
        filepaths = list(self.get_pic_filepaths(directories))
        return LeftImageLoader(filepaths)

In [ ]:
class OutsideTopDataset(torch.utils.data.IterableDataset):
    
    def __init__(self):
        super(OutsideTopDataset).__init__()
        self.dataset = self.load_dataset()
                
    def __iter__(self):
        for x, _ in iter(self.dataset):
            both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = -0.1)
            ])

            xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
            yonly = MaskedAreaTransform(config)
            p = both(x)
            x_, y_ = xonly(p), yonly(p)
            yield x_, y_
            
    def __len__(self):
        return len(self.train_dataset)
    
    def get_pic_filepaths(self, directions):
        for direction in directions:
            for pic in os.listdir(f"OutsideData/Selected Options/{direction}"):
                yield os.path.join(f"OutsideData/Selected Options/{direction}", pic)
                
    def load_dataset(self):
        directories = ["Top"]
        filepaths = list(self.get_pic_filepaths(directories))
        return TopImageLoader(filepaths)

In [ ]:
top_loader = OutsideTopDataset().dataset
right_loader = OutsideRightDataset().dataset
left_loader = OutsideLeftDataset().dataset

In [ ]:
both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = -0.1)
            ])
xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
yonly = MaskedAreaTransform(config)

for i, (X, y) in enumerate(top_loader):
    p = both(X)
    x_, y_ = xonly(p), yonly(p)
    x_, y_ = torch.reshape(x_, (1, x_.shape[0], x_.shape[1], x_.shape[2])), torch.reshape(y_, (1, y_.shape[0], y_.shape[1], y_.shape[2]))
    x_ = x_.to(torch.device('cuda'))
    print(x_.device)
    break
    
u.show(x_)
print(x_.shape)

In [ ]:
u.show(gen(x_))